In [4]:
import json
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN, GRU, LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.layers import Convolution1D, MaxPooling1D
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, precision_recall_curve, auc, roc_curve, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn import preprocessing
from sklearn import cross_validation
from sklearn.metrics import log_loss
import itertools
import ast
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import sys, os
import traceback
from operator import itemgetter

import progressbar

In [5]:
def read_data(rootdir):
    data_list = []
    for subdir, dirs, files in os.walk(rootdir):
        for fname in files:
            #print os.path.join(subdir, file)
            filepath = subdir + os.sep + fname
            with open(filepath) as f:
                for line1,line2 in itertools.izip_longest(*[f]*2):
                    try:
                        text = ast.literal_eval(line1)
                        label = ast.literal_eval(line2)
                        if len(text) > 3:
                            sentence = zip(text, label)
                            sentence = [item for item in sentence if item[0].strip() != '']
                            data_list.append(sentence)
                    except:
                        pass
            f.close()
    return data_list

In [6]:
data_list_raw = read_data('../data/input/GO/')

#raw data
print data_list_raw[0]
print len(data_list_raw)

data_list = []
for each in data_list_raw:
    sent = set(map(itemgetter(1), each))
    if len(sent) != 1:
        data_list.append(each)
    
#clean data
print data_list[0]
print len(data_list)

train_test_cutoff = int(.70 * len(data_list)) 
training_sentences = data_list[:train_test_cutoff]
testing_sentences = data_list[train_test_cutoff:]
 
train_val_cutoff = int(.25 * len(training_sentences))
validation_sentences = training_sentences[:train_val_cutoff]
training_sentences = training_sentences[train_val_cutoff:]

[('A', 'NA'), ('Hybrid', 'NA'), ('Photoreceptor', 'NA'), ('Expressing', 'NA'), ('Both', 'NA'), ('Rod', 'NA'), ('and', 'NA'), ('Cone', 'NA'), ('Genes', 'NA'), ('in', 'NA'), ('a', 'NA'), ('Mouse', 'NA'), ('Model', 'NA'), ('of', 'NA'), ('Enhanced', 'NA'), ('SCone', 'NA'), ('Syndrome', 'NA')]
5065
[('Rod', 'NA'), ('and', 'NA'), ('cone', 'NA'), ('photoreceptors', 'NA'), ('subserve', 'NA'), ('vision', 'NA'), ('under', 'NA'), ('dim', 'NA'), ('and', 'NA'), ('bright', 'NA'), ('light', 'NA'), ('conditions', 'NA'), ('respectively', 'NA'), ('The', 'NA'), ('differences', 'NA'), ('in', 'NA'), ('their', 'NA'), ('function', 'NA'), ('are', 'NA'), ('thought', 'NA'), ('to', 'NA'), ('stem', 'NA'), ('from', 'NA'), ('their', 'NA'), ('different', 'NA'), ('gene', 'NA'), ('expression', 'NA'), ('patterns', 'NA'), ('morphologies', 'NA'), ('and', 'NA'), ('synaptic', 'GO:0045202'), ('connectivities', 'NA'), ('In', 'NA'), ('this', 'NA'), ('study', 'NA'), ('we', 'NA'), ('have', 'NA'), ('examined', 'NA'), ('the', 'NA

In [7]:
print training_sentences[0]

[('Horizontal', 'NA'), ('sections', 'NA'), ('of', 'NA'), ('the', 'NA'), ('indicated', 'NA'), ('genotypes', 'NA'), ('and', 'NA'), ('ages', 'NA'), ('E14', 'NA'), ('and', 'NA'), ('E16', 'NA'), ('the', 'NA'), ('period', 'NA'), ('during', 'NA'), ('which', 'NA'), ('SACs', 'NA'), ('are', 'NA'), ('born', 'NA'), ('were', 'NA'), ('stained', 'NA'), ('for', 'NA'), ('nuclei', 'GO:0005634'), ('DAPI', 'NA'), ('blue', 'NA'), ('and', 'NA'), ('either', 'NA'), ('Sphase', 'NA'), ('upper', 'NA'), ('two', 'NA'), ('panels', 'NA'), ('antiBrdU', 'NA'), ('red', 'NA'), ('or', 'NA'), ('apoptosis', 'NA'), ('lower', 'NA'), ('two', 'NA'), ('panels', 'NA'), ('TUNEL', 'NA'), ('red', 'NA'), ('In', 'NA'), ('Rb\xe2\x88\x92\xe2\x88\x92', 'NA'), ('retinas', 'NA'), ('BrdU', 'NA'), ('and', 'NA'), ('TUNEL', 'NA'), ('cells', 'GO:0005623'), ('can', 'NA'), ('be', 'NA'), ('seen', 'NA'), ('in', 'NA'), ('the', 'NA'), ('inner', 'NA'), ('retina', 'NA'), ('arrows', 'NA'), ('Inactivation', 'NA'), ('of', 'NA'), ('E2f1', 'NA'), ('rescued

In [8]:
def add_basic_features(sentence_terms, index):
    """ Compute some very basic word features.
 
        :param sentence_terms: [w1, w2, ...] 
        :type sentence_terms: list
        :param index: the index of the word 
        :type index: int
        :return: dict containing features
        :rtype: dict
    """

    term = sentence_terms[index]

    return {
        'nb_terms': len(sentence_terms),
        'term': term,
        'is_first': index == 0,
        'is_last': index == len(sentence_terms) - 1,
        'is_capitalized': term[0].upper() == term[0],
        'is_all_caps': term.upper() == term,
        'is_all_lower': term.lower() == term,
        'prefix-1': term[0],
        'prefix-2': term[:2],
        'prefix-3': term[:3],
        'suffix-1': term[-1],
        'suffix-2': term[-2:],
        'suffix-3': term[-3:],
        'prev_word': '' if index == 0 else sentence_terms[index - 1],
        'next_word': '' if index == len(sentence_terms) - 1 else sentence_terms[index + 1]
    }

In [9]:
def untag(tagged_sentence):
    """ 
    Remove the tag for each tagged term. 
 
    :param tagged_sentence: a POS tagged sentence
    :type tagged_sentence: list
    :return: a list of tags
    :rtype: list of strings
    """
    return [w for w, _ in tagged_sentence]
 
def transform_to_dataset(tagged_sentences):
    """
    Split tagged sentences to X and y datasets and append some basic features.
 
    :param tagged_sentences: a list of POS tagged sentences
    :param tagged_sentences: list of list of tuples (term_i, tag_i)
    :return: 
    """
    X, y = [], []
 
    for pos_tags in tagged_sentences:
        for index, (term, class_) in enumerate(pos_tags):
            # Add basic NLP features for each sentence term
            X.append(add_basic_features(untag(pos_tags), index))
            y.append(class_)
    return X, y

In [10]:
X_train, y_train = transform_to_dataset(training_sentences)
X_test, y_test = transform_to_dataset(testing_sentences)
X_val, y_val = transform_to_dataset(validation_sentences)

In [11]:
from sklearn.feature_extraction import DictVectorizer
 
# Fit our DictVectorizer with our set of features
dict_vectorizer = DictVectorizer(sparse=False)
dict_vectorizer.fit(X_train + X_test + X_val)

DictVectorizer(dtype=<type 'numpy.float64'>, separator='=', sort=True,
        sparse=False)

In [12]:
# Convert dict features to vectors
X_train = dict_vectorizer.transform(X_train)
X_test = dict_vectorizer.transform(X_test)
X_val = dict_vectorizer.transform(X_val)

In [13]:
# Fit LabelEncoder with our list of classes
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(y_train + y_test + y_val)

LabelEncoder()

In [14]:
# Encode class values as integers
y_train = label_encoder.transform(y_train)
y_test = label_encoder.transform(y_test)
y_val = label_encoder.transform(y_val)

In [15]:
# Convert integers to dummy variables (one hot encoded)
from keras.utils import np_utils
 
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
y_val = np_utils.to_categorical(y_val)

print X_train.shape, X_test.shape, y_train.shape, y_test.shape

(107916, 63295) (63687, 63295) (107916, 104) (63687, 104)


In [ ]:
#simple test
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)
clf_probs = clf.predict_proba(X_test)
print clf.score(X_test, y_test)   

In [ ]:
clf = RandomForestClassifier(n_estimators=100)

cv = ShuffleSplit(n_splits=10, test_size=0.3, random_state=0)

train_errors = []
test_errors = []

scores = []
roc_scores = []
fprs, tprs = [], []

pr_scores = []
precisions, recalls, thresholds = [], [], []

clfs = []

for train, test in cv.split(X):
    X_train, Y_train = X[train], Y[train]
    X_test, Y_test =  X[test], Y[test]

print "Creating Classifier"
clf.fit(X_train, Y_train)
print "Classifier Complete"

train_score = clf.score(X_train, Y_train)
test_score = clf.score(X_test, Y_test)

train_errors.append(1 - train_score)
test_errors.append(1 - test_score)

scores.append(test_score)
proba = clf.predict_proba(X_test)

label_idx = 1
fpr, tpr, roc_thresholds = roc_curve(Y_test, proba[:, label_idx])
precision, recall, pr_thresholds = precision_recall_curve(Y_test, proba[:, label_idx])

roc_scores.append(auc(fpr, tpr))
fprs.append(fpr)
tprs.append(tpr)

pr_scores.append(auc(recall,precision))
precisions.append(precision)
recalls.append(recall)
thresholds.append(pr_thresholds)

print(classification_report(Y_test, proba[:, label_idx] > 0.5))

# get medium clone
scores_to_sort = pr_scores  # roc_scores
medium = np.argsort(scores_to_sort)[len(scores_to_sort) / 2]

print "Mean Accuracy: ", str(np.mean(scores))
print "Mean ROC Score: ", str(np.mean(roc_scores))
print "Mean PR Score: ", str(np.mean(pr_scores))
